## Setup notebook

In [0]:
import pandas as pd
import numpy as np
import scipy.spatial
import re
import warnings
import logging

## Load in locations that have already been through geolocation and now we want to create a read-across with an internal location grouping called 'RDI Lens'

In [0]:
df = pd.read_csv("/final_df_geolocated.csv")
df.head(5)

,location,geocoded_address,geocoding_function,"latitude, longitude",matchcode,matchtype,LocationIQ_potential_locations,name,house_number,road,neighbourhood,suburb,island,city,county,state,state_code,postcode,country,country_code,match accuracy
0,London,"{'latitude': '51.5074456', 'longitude': '-0.1277653', 'matchcode':...",within countrycodes,"51.5074456, -0.1277653",exact,centroid,8,No address element,No address element,No address element,No address element,No address element,No address element,London,No address element,England,No address element,No address element,United Kingdom,gb,high accuracy
1,"London, GB, W1A 1AA","{'latitude': '51.5074456', 'longitude': '-0.1277653', 'matchcode':...",within countrycodes,"51.5074456, -0.1277653",exact,centroid,4,No address element,No address element,No address element,No address element,No address element,No address element,London,No address element,England,No address element,No address element,United Kingdom,gb,high accuracy
2,"LUTON AIRPORT, UK","{'latitude': '51.87810155', 'longitude': '-0.36642547440865236', '...",within countrycodes,"51.87810155, -0.36642547440865236",approximate,point,2,London Luton Airport,No address element,Chiltern Green,No address element,Hyde,No address element,Chiltern Green,Central Bedfordshire,England,No address element,LU2 9LY,United Kingdom,gb,low accuracy
3,"NORTHOLT, UK","{'latitude': '51.5484582', 'longitude': '-0.3695247', 'matchcode':...",within countrycodes,"51.5484582, -0.3695247",approximate,point,6,Northolt,No address element,Mandeville Road,No address element,No address element,No address element,London,No address element,England,No address element,UB5 4AA,United Kingdom,gb,low accuracy
4,United Kingdom,"{'latitude': '54.7023545', 'longitude': '-3.2765753', 'matchcode':...",within countrycodes,"54.7023545, -3.2765753",exact,centroid,2,No address element,No address element,No address element,No address element,No address element,No address element,No address element,No address element,No address element,No address element,No address element,United Kingdom,gb,high accuracy


### Pull in a pre-constructed table that acts as a read-across, allowing us to allocate every geolocated UK location a RDI lens grouping
We use this second dataframe as part of a .merge() to pull-across the 'Area/City Code' to the original df dataframe

In [0]:
uk_city_grouping = pd.read_csv("/uk_rdi_grouping.csv")
uk_city_grouping.head(5)

,RDI City,"latitude, longitude",RDI grouping - region/nation,Area/City Code,Country,Region,Country Code,Currency Code,Currency Name
0,London,"51.5074456, -0.1277653",London,GB001/GB002,United Kingdom,Western Europe,GBR,GBP,United Kingdom Pound
1,Westminster,"51.5004439, -0.1265398",London,GB001/GB002,United Kingdom,Western Europe,GBR,GBP,United Kingdom Pound
2,Bedfordshire,"52.064003400000004, -0.41111375560310925",South East,GB003,United Kingdom,Western Europe,GBR,GBP,United Kingdom Pound
3,Berkshire,"51.453488899999996, -1.0318729593399247",South East,GB003,United Kingdom,Western Europe,GBR,GBP,United Kingdom Pound
4,Brighton & Hove,"50.8214626, -0.1400561",South East,GB003,United Kingdom,Western Europe,GBR,GBP,United Kingdom Pound


## To have a successfull .merge(), we need all rows within df to have a key (city)
But not all locations have a 'city'. For those that do not, we are allocating them an **"approximate city" based on their latitutde & longitude, matching to the nearest city**

In [0]:
# Add a blank column called 'proximity allocated' to keep track of which ones were allocated a city
df["proximity allocated"] = ""

# We're only looking at rows that 1. have a lat & lon and 2. don't currently have a 'city'
filtered_df = df[
    (df["latitude, longitude"] != "No latitude, No longitude")
    & (df["city"] == "No address element")
]

# Create KD-Tree using dataframe 'uk_city_grouping'
latitude_longitude = uk_city_grouping["latitude, longitude"].apply(
    lambda x: list(map(float, x.split(", ")))
)
tree = scipy.spatial.KDTree(np.array(list(latitude_longitude)))

# Loop through each row in the filtered dataframe
for index, row in filtered_df.iterrows():
    # Get the current 'latitude, longitude' combination
    current_lat, current_lng = list(map(float, row["latitude, longitude"].split(", ")))

    # Find the nearest city using the KD-Tree
    distance, nearest_index = tree.query([[current_lat, current_lng]])

    # Get the nearest city's name
    nearest_city = uk_city_grouping.iloc[nearest_index]["RDI City"].values[0]

    # Update the 'city' column in the 'df' dataframe
    df.at[index, "city"] = nearest_city
    # Include mention that this RDI grouping will be based on a approximated city, not the real allocated city by LocationIQ
    df.at[index, "proximity allocated"] = "yes"


df.head(5)

,location,geocoded_address,geocoding_function,"latitude, longitude",matchcode,matchtype,LocationIQ_potential_locations,name,house_number,road,neighbourhood,suburb,island,city,county,state,state_code,postcode,country,country_code,match accuracy,proximity allocated
0,London,"{'latitude': '51.5074456', 'longitude': '-0.1277653', 'matchcode':...",within countrycodes,"51.5074456, -0.1277653",exact,centroid,8,No address element,No address element,No address element,No address element,No address element,No address element,London,No address element,England,No address element,No address element,United Kingdom,gb,high accuracy,
1,"London, GB, W1A 1AA","{'latitude': '51.5074456', 'longitude': '-0.1277653', 'matchcode':...",within countrycodes,"51.5074456, -0.1277653",exact,centroid,4,No address element,No address element,No address element,No address element,No address element,No address element,London,No address element,England,No address element,No address element,United Kingdom,gb,high accuracy,
2,"LUTON AIRPORT, UK","{'latitude': '51.87810155', 'longitude': '-0.36642547440865236', '...",within countrycodes,"51.87810155, -0.36642547440865236",approximate,point,2,London Luton Airport,No address element,Chiltern Green,No address element,Hyde,No address element,Chiltern Green,Central Bedfordshire,England,No address element,LU2 9LY,United Kingdom,gb,low accuracy,
3,"NORTHOLT, UK","{'latitude': '51.5484582', 'longitude': '-0.3695247', 'matchcode':...",within countrycodes,"51.5484582, -0.3695247",approximate,point,6,Northolt,No address element,Mandeville Road,No address element,No address element,No address element,London,No address element,England,No address element,UB5 4AA,United Kingdom,gb,low accuracy,
4,United Kingdom,"{'latitude': '54.7023545', 'longitude': '-3.2765753', 'matchcode':...",within countrycodes,"54.7023545, -3.2765753",exact,centroid,2,No address element,No address element,No address element,No address element,No address element,No address element,Cumbria,No address element,No address element,No address element,No address element,United Kingdom,gb,high accuracy,yes


## Now that all rows have a city value, we can merge

In [0]:
merged_df_city = df.merge(
    uk_city_grouping[["RDI City", "RDI grouping - region/nation", "Area/City Code"]],
    left_on="city",
    right_on="RDI City",
    how="left",
)

missing_city = merged_df_city['RDI City'].isnull().sum()
total_rows = merged_df_city.shape[0]
missing_city_perc = (missing_city/total_rows) * 100
print(f"There are {missing_city} locations missing a match ({missing_city_perc:.1f}%)")

There are 548 locations missing a match (59.1%)


## Some locations remain unmatched!
This occurs when address elements are misclassified (e.g., a state is used instead of a city). We address this by substituting the 'city' key with alternative address elements—starting with 'county'—that the API might have confused.

In [0]:
null_mask = merged_df_city["RDI City"].isnull()
merged_df_county_null = merged_df_city.loc[null_mask]

# Update the 'merged_df_city' dataframe by removing the rows where 'location RDI' is null
merged_df_city = merged_df_city.loc[~null_mask]

merged_df_county_null.drop(
    ["RDI City", "RDI grouping - region/nation", "Area/City Code"], axis=1, inplace=True
)

# using .merge to check for matches of 'county' to 'RDI City'
merged_df_county = merged_df_county_null.merge(
    uk_city_grouping[["RDI City", "RDI grouping - region/nation", "Area/City Code"]],
    left_on="county",
    right_on="RDI City",
    how="left",
)

missing_county = merged_df_county['RDI City'].isnull().sum()
non_missing_county = merged_df_county.shape[0] - missing_county
missing_county_perc = (missing_county/total_rows) * 100
print(f"Matching by 'county' instead of 'city' has generated {non_missing_county} more location matches.")
print(f"There are still {missing_county} locations missing a match ({missing_county_perc:.1f}%.)")

Matching by 'county' instead of 'city' has generated 192 more location matches.
There are still 356 locations missing a match (38.4%.)


## Let's continue with 'state' instead of 'city' or 'county'

In [0]:
null_mask_state = merged_df_county["RDI City"].isnull()
merged_df_state_null = merged_df_county.loc[null_mask_state]

# Update the 'merged_df_county' dataframe by removing the rows where 'RDI City' is null
merged_df_county = merged_df_county.loc[~null_mask_state]

merged_df_state_null.drop(
    ["RDI City", "RDI grouping - region/nation", "Area/City Code"], axis=1, inplace=True
)

# using .merge to check for matches of 'state' to 'RDI City'
merged_df_state = merged_df_state_null.merge(
    uk_city_grouping[["RDI City", "RDI grouping - region/nation", "Area/City Code"]],
    left_on="state",
    right_on="RDI City",
    how="left",
)

missing_state = merged_df_state['RDI City'].isnull().sum()
non_missing_state = merged_df_state.shape[0] - missing_state
missing_state_perc = (missing_state/total_rows) * 100
print(f"Matching by 'state' instead of 'county' or 'city' has generated {non_missing_state} more location matches.")
print(f"There are still {missing_state} locations missing a match ({missing_state_perc:.1f}%).")

Matching by 'state' instead of 'county' or 'city' has generated 83 more location matches.
There are still 273 locations missing a match (29.4%).


## Final Matching Method
After all previous matching attempts, some locations—often small towns or villages not included in our read-across table—remain unmatched. As the final step, we apply proximity matching to force a match, ensuring comprehensive coverage.

In [0]:
uk_merged_df = pd.concat([merged_df_city, merged_df_county, merged_df_state])
df_nomatch = uk_merged_df[uk_merged_df["RDI City"].isnull()].copy()

# Drop rows from uk_merged_df where 'RDI City' is null
uk_merged_df = uk_merged_df.dropna(subset=["RDI City"])

# Loop through each row in the filtered dataframe
for index, row in df_nomatch.iterrows():
    # Get the current 'latitude, longitude' combination
    current_lat, current_lng = list(map(float, row["latitude, longitude"].split(", ")))

    # Find the nearest city using the KD-Tree
    distance, nearest_index = tree.query([[current_lat, current_lng]])

    # Get the nearest city's name
    nearest_city = uk_city_grouping.iloc[nearest_index]["RDI City"].values[0]

    # Update the 'city' column in the 'df_nomatch' dataframe
    df_nomatch.at[index, "city"] = nearest_city
    # Include mention that this RDI grouping will be based on a approximated city, not the real allocated city by LocationIQ
    df_nomatch.at[index, "proximity allocated"] = "yes"

df_nomatch.drop(
    ["RDI City", "RDI grouping - region/nation", "Area/City Code"], axis=1, inplace=True
)

df_nomatch = df_nomatch.merge(
    uk_city_grouping[["RDI City", "RDI grouping - region/nation", "Area/City Code"]],
    left_on="city",
    right_on="RDI City",
    how="left",
)

if df_nomatch['RDI City'].isnull().any():
    print("Still some errors")
else:
    print("All locations now matched")

All locations now matched


## Creating the final table with all rows now matched
and double-checking our data!

In [0]:
# Add all values in 'df_nomatch' to 'uk_merged_df' and now all locations will have a RDI read-across
final_merged_df = pd.concat([uk_merged_df, df_nomatch])

# Automatically fill in column 'proximity allocated' with 'no' where currently not 'yes'
final_merged_df.loc[
    final_merged_df["proximity allocated"] != "yes", "proximity allocated"
] = "no"

# Handling of locations that just say the country
country_names = ["United Kingdom", ", United Kingdom"]

final_merged_df.loc[
    final_merged_df["location"].isin(country_names),
    ["RDI grouping - region/nation", 
     "RDI City", 
     "Area/City Code"],
] = "COUNTRY NULL"


# 1. Do we end up with the same number of rows that we started with?
final_rows = final_merged_df.shape[0]
if final_rows == total_rows:
    print("1. Number of rows as expected")
else:
    print(f"1. WARNING: More/Less rows than expected")

# 2. Does every row have an RDI grouping?
if final_merged_df["RDI grouping - region/nation"].isnull().sum() == 0:
    print("2. Every row has an RDI Grouping")
else:
    print(f"2. WARNING: Not every row has an RDI Grouping")

# 3. Check for duplicate rows
duplicate_rows = final_merged_df.duplicated().sum()
if duplicate_rows == 0:
    print("3. No duplicate rows found")
else:
    print(f"3. WARNING: Found {duplicate_rows} duplicate rows")

1. Number of rows as expected
2. Every row has an RDI Grouping
3. No duplicate rows found


## With our location data now structured and aligned with our RDI Lens grouping system, the final step is to map these locations back to the original job IDs and perform our analyses!